In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# SubmodelComp

`SubmodelComp` provides a way to evaluate an OpenMDAO system within an OpenMDAO system.

# SubmodelComp Options

In [ ]:
import openmdao.api as om

subprob = om.Problem()
submodel = om.Group()
submodel.add_subsystem('subComp', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

subprob.model.add_subsystem('submodel', submodel)

om.show_options_table(om.SubmodelComp(problem=subprob,
                                      inputs=['r', 'theta'],
                                      outputs=['x']))

# SubmodelComp Constructor

The call signature for the `SubmodelComp` constructor is

```{eval-rst}
    .. automethod:: openmdao.components.subproblem_comp.SubmodelComp.__init__
        :noindex:
```

# Using the SubmodelComp

`SubmodelComp` allows you to add a component that is a `System` itself. The only required argument is `problem`. This takes an instantiated problem that already has a model. That way, the user can declare any desired options for the problem and have it used as for the underlying subproblem.

Other arguments include `inputs`, `outputs`, and `reports`. Input and output argument formats are discussed in the next section, but if nothing is passed in `SubmodelComp`'s instantiation, then the inputs and outputs will have to be added later using `add_input` and `add_output`. The `reports` option is there for underlying subproblem. `SubmodelComp` performs more efficiently if it doesn't have to write reports, but if reports are desired, then the `reports` argument can be switched to `True`.

`SubmodelComp` provides a way to evaluate an OpenMDAO system within an OpenMDAO system. Unlike a typical subsystem, using a subproblem exposes only a limited number of inputs and outputs of the underlying model to the parent system. This may be beneficial from a performance standpoint when there are many inputs and outputs of the internal system that aren't needed in the external system.

Note that this means that nonlinear and linear solvers are needed on the subproblem's model if implicit behavior is present, since the internal variables are invisible to solvers in the parent system.

At this time, OpenMDAO does not evaluate gradients through the optimization process, so a gradient-based driver at the outer level cannot access derivatives across the optimization on the inner level.

## IO Format

Inputs and outputs can be passed into `SubmodelComp` as a list of `str`, `tuple`, or both. If the list element is `str`, then it must be the group level promoted name that is passed. For this, the IO to be referred to must be promoted to its group level. If the list element is `tuple`, then the first element of the `tuple` must be the group level promoted name or the absolute name of the variable, and the second element is the alias that `SubmodelComp` will refer to it as.

e.g. (path.to.var, desired_var_name)  
or (var, desired_var_name) if the variable is already promoted at the group level

Input and output names can be wildcards too, only if the type is `str`.

If no IO is given in its instantiation, inputs and outputs will have to be declared using the `add_input` and `add_output` methods.

# Example: Implementing SubmodelComp

In [ ]:
import openmdao.api as om
from numpy import pi

p = om.Problem()

model = om.Group()
model.add_subsystem('supComp', om.ExecComp('z = x**2 + y'),
                    promotes_inputs=['x', 'y'],
                    promotes_outputs=['z'])

submodel1 = om.Group()
submodel1.add_subsystem('sub1_ivc_r', om.IndepVarComp('r', 1.),
                        promotes_outputs=['r'])
submodel1.add_subsystem('sub1_ivc_theta', om.IndepVarComp('theta', pi),
                        promotes_outputs=['theta'])
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

submodel2 = om.Group()
submodel2.add_subsystem('sub2_ivc_r', om.IndepVarComp('r', 2),
                        promotes_outputs=['r'])
submodel2.add_subsystem('sub2_ivc_theta', om.IndepVarComp('theta', pi/2),
                        promotes_outputs=['theta'])
submodel2.add_subsystem('subComp2', om.ExecComp('y = r*sin(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['y'])

subprob1 = om.Problem()
subprob1.model.add_subsystem('submodel1', submodel1)
subprob2 = om.Problem()
subprob2.model.add_subsystem('submodel2', submodel2)

subcomp1 = om.SubmodelComp(problem=subprob1,
                            inputs=['r', 'theta'], outputs=['x'])
subcomp2 = om.SubmodelComp(problem=subprob2,
                            inputs=['r', 'theta'], outputs=['y'])

p.model.add_subsystem('sub1', subcomp1, promotes_inputs=['r','theta'],
                            promotes_outputs=['x'])
p.model.add_subsystem('sub2', subcomp2, promotes_inputs=['r','theta'],
                            promotes_outputs=['y'])
p.model.add_subsystem('supModel', model, promotes_inputs=['x','y'],
                            promotes_outputs=['z'])

p.model.set_input_defaults('r', 1)
p.model.set_input_defaults('theta', pi)

p.setup(force_alloc_complex=True)

p.run_model()
cpd = p.check_partials(method='cs', out_stream=None)

# Example: Variable Aliases

In [ ]:
import openmdao.api as om

p = om.Problem()
model = om.Group()

model.add_subsystem('subsys', om.ExecComp('z = x**2 + y**2'))
subprob = om.Problem()
subprob.model.add_subsystem('submodel', model)
subcomp = om.SubmodelComp(problem=subprob,
                            inputs=[('subsys.x', 'a'), ('subsys.y', 'b')],
                            outputs=[('subsys.z', 'c')])

p.model.add_subsystem('subcomp', subcomp, promotes_inputs=['a', 'b'], promotes_outputs=['c'])
p.setup()

p.set_val('a', 1)
p.set_val('b', 2)

p.run_model()

p.run_model()

# Example: Using Variables with the Same Name

In [ ]:
import openmdao.api as om

p = om.Problem()

model = om.Group()

model.add_subsystem('x1Comp', om.ExecComp('x1 = x*3'))
model.add_subsystem('x2Comp', om.ExecComp('x2 = x**3'))
model.connect('x1Comp.x1', 'model.x1')
model.connect('x2Comp.x2', 'model.x2')
model.add_subsystem('model', om.ExecComp('z = x1**2 + x2**2'))

subprob = om.Problem()
subprob.model.add_subsystem('submodel', model)

comp = om.SubmodelComp(problem=subprob, 
                        inputs=[('x1Comp.x', 'x'), ('x2Comp.x', 'y')],
                        outputs=[('model.z', 'z')])

p.model.add_subsystem('comp', comp)

p.model.set_input_defaults('comp.x', 1)
p.model.set_input_defaults('comp.y', 2)

p.setup()

p.run_model()

# Example: Using Wildcard Variables

In [ ]:
import openmdao.api as om

p = om.Problem()
model = om.Group()

model.add_subsystem('sub', om.ExecComp('z = x1**2 + x2**2 + x3**2'), promotes=['*'])
subprob = om.Problem()
subprob.model.add_subsystem('submodel', model)
comp = om.SubmodelComp(problem=subprob, inputs=['x*'], outputs=['*'])

p.model.add_subsystem('comp', comp, promotes_inputs=['*'], promotes_outputs=['*'])
p.setup()

p.set_val('x1', 1)
p.set_val('x2', 2)
p.set_val('x3', 3)

p.run_model()

# Example: Adding Inputs and Outputs Before Setup

In [ ]:
import openmdao.api as om

p = om.Problem()

model = om.Group()
model.add_subsystem('supComp', om.ExecComp('z = x**2 + y'),
                    promotes_inputs=['x', 'y'],
                    promotes_outputs=['z'])

submodel1 = om.Group()
submodel1.add_subsystem('sub1_ivc_r', om.IndepVarComp('r', 1.),
                        promotes_outputs=['r'])
submodel1.add_subsystem('sub1_ivc_theta', om.IndepVarComp('theta', pi),
                        promotes_outputs=['theta'])
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'))

submodel2 = om.Group()
submodel2.add_subsystem('sub2_ivc_r', om.IndepVarComp('r', 2),
                        promotes_outputs=['r'])
submodel2.add_subsystem('sub2_ivc_theta', om.IndepVarComp('theta', pi/2),
                        promotes_outputs=['theta'])
submodel2.add_subsystem('subComp2', om.ExecComp('y = r*sin(theta)'))

subprob1 = om.Problem()
subprob1.model.add_subsystem('submodel1', submodel1)
subprob2 = om.Problem()
subprob2.model.add_subsystem('submodel2', submodel2)

comp1 = om.SubmodelComp(problem=subprob1)
comp2 = om.SubmodelComp(problem=subprob2)

comp1.add_input('subComp1.r', name='r')
comp1.add_input('subComp1.theta', name='theta')
comp2.add_input('subComp2.r', name='r')
comp2.add_input('subComp2.theta', name='theta')

comp1.add_output('subComp1.x', name='x')
comp2.add_output('subComp2.y', name='y')

p.model.add_subsystem('sub1', comp1, promotes_inputs=['r','theta'],
                            promotes_outputs=['x'])
p.model.add_subsystem('sub2', comp2, promotes_inputs=['r','theta'],
                            promotes_outputs=['y'])
p.model.add_subsystem('supModel', model, promotes_inputs=['x','y'],
                            promotes_outputs=['z'])

p.model.set_input_defaults('r', 1)
p.model.set_input_defaults('theta', pi)

p.setup(force_alloc_complex=True)

p.run_model()
cpd = p.check_partials(method='cs', out_stream=None)

# Example: Adding Inputs and Outputs in Configure

In [ ]:
import openmdao.api as om

class Subsys1(om.Group):
    def setup(self):
        model = om.Group()
        comp = om.ExecComp('x = r*cos(theta)')
        model.add_subsystem('comp', comp, promotes_inputs=['r', 'theta'],
                            promotes_outputs=['x'])
        subprob = om.Problem(); subprob.model.add_subsystem('model', model)
        self.add_subsystem('submodel1', om.SubmodelComp(problem=subprob))

    def configure(self):
        self._get_subsystem('submodel1').add_input('r')
        self._get_subsystem('submodel1').add_input('theta')
        self._get_subsystem('submodel1').add_output('x')

        self.promotes('submodel1', ['r', 'theta', 'x'])

class Subsys2(om.Group):
    def setup(self):
        model = om.Group()
        comp = om.ExecComp('y = r*sin(theta)')
        model.add_subsystem('comp', comp, promotes_inputs=['r', 'theta'],
                            promotes_outputs=['y'])
        subprob = om.Problem(); subprob.model.add_subsystem('model', model)
        self.add_subsystem('submodel2', om.SubmodelComp(problem=subprob))

    def configure(self):
        self._get_subsystem('submodel2').add_input('r')
        self._get_subsystem('submodel2').add_input('theta')
        self._get_subsystem('submodel2').add_output('y')

        self.promotes('submodel2', ['r', 'theta', 'y'])

p = om.Problem()

model = om.Group()
model.add_subsystem('supComp', om.ExecComp('z = x**2 + y'),
                    promotes_inputs=['x', 'y'],
                    promotes_outputs=['z'])

p.model.add_subsystem('sub1', Subsys1(), promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])
p.model.add_subsystem('sub2', Subsys2(), promotes_inputs=['r', 'theta'],
                        promotes_outputs=['y'])
p.model.add_subsystem('supModel', model, promotes_inputs=['x','y'],
                        promotes_outputs=['z'])

p.setup(force_alloc_complex=True)

p.set_val('r', 1)
p.set_val('theta', pi)

p.run_model()
cpd = p.check_partials(method='cs', out_stream=None)